In [251]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *

import numpy as np

In [252]:
mesh = Mesh(unit_square.GenerateMesh(maxh=1))

order = 1
fes = HCurl(mesh, order=order, type1=False)
gfu = GridFunction(fes)
gfut = GridFunction(fes, multidim=0)

Here a little testing to see what happens with type1=True and type1=False.
My VSCode Drawing of gridfunctions is very buggy, but it seems like that type1=False leads to the interior of the elements having a function as well.
This because first of all type1=False also leads to the DOF's increasing and one can see it visually.

In [253]:
print(fes.ndof)
settings = {"camera": {"transformations": [
        {"type": "rotateX", "angle": -45}]}}

# for i in range(len(gfu.vec)):

#     gfu.vec[:] = 0
#     gfu.vec[i] = 1

#     gfut.AddMultiDimComponent(gfu.vec)

# Draw(gfut, deformation=False, animate=True, autoscale = True , order = 3,  vectors = { "grid_size":30}, settings=settings);

10


I wanna add testing to see what happens to the crossproduct in 2D, I need this to nicely write down my boundary integrals independent of the dimension

In [254]:
normal = 1*specialcf.normal(mesh.dim)
tangent = 1*specialcf.tangential(mesh.dim)
# z_vector = Cross(normal, tangent) # Doesnt work bcs of the dimensions!, obviously, but thought maybe it works and yields a scalar..

# Draw(z_vector)

Wanna Test some mesh size calculation thingies!

In [266]:
# Helper function to calculate the distance between two points in 2D or 3D
def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

# Helper function to calculate the area of a triangle in 2D using Heron's formula
def triangle_area(a, b, c):
    s = (a + b + c) / 2  # semi-perimeter
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))
    return area

# Helper function to calculate the circumradius of a triangle in 2D
def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

# Helper function to calculate the volume of a tetrahedron
def tetrahedron_volume(vertices):
    # Form a matrix with vertex differences (relative to V0)
    matrix = np.array([
        [vertices[1][0] - vertices[0][0], vertices[1][1] - vertices[0][1], vertices[1][2] - vertices[0][2]],
        [vertices[2][0] - vertices[0][0], vertices[2][1] - vertices[0][1], vertices[2][2] - vertices[0][2]],
        [vertices[3][0] - vertices[0][0], vertices[3][1] - vertices[0][1], vertices[3][2] - vertices[0][2]]
    ])
    # Compute the volume as the absolute value of the determinant divided by 6
    volume = np.abs(np.linalg.det(matrix)) / 6.0
    return volume

# Helper function to calculate the circumradius of a tetrahedron using the Cayley-Menger determinant
def circumradius_3D(vertices):
    # Calculate all six edge lengths
    edge_lengths = [edge_length(vertices[i], vertices[j], 3) for i in range(4) for j in range(i+1, 4)]
    
    # Construct Cayley-Menger matrix
    cm_matrix = np.array([
        [0, 1, 1, 1, 1],
        [1, 0, edge_lengths[0]**2, edge_lengths[1]**2, edge_lengths[2]**2],
        [1, edge_lengths[0]**2, 0, edge_lengths[3]**2, edge_lengths[4]**2],
        [1, edge_lengths[1]**2, edge_lengths[3]**2, 0, edge_lengths[5]**2],
        [1, edge_lengths[2]**2, edge_lengths[4]**2, edge_lengths[5]**2, 0]
    ])
    
    # Calculate determinant
    cm_determinant = np.linalg.det(cm_matrix)
    
    # Calculate volume of the tetrahedron
    volume = tetrahedron_volume(vertices)
    
    # Return circumradius or handle degenerate case
    if cm_determinant <= 0 or volume == 0:
        print("Warning: Degenerate or invalid tetrahedron.")
        return np.inf  # Large value for degenerate cases
    
    circumradius = np.sqrt(cm_determinant) / (24 * volume)
    return circumradius

# Main function to calculate the minimum characteristic length 'h' for a 2D or 3D mesh
def calc_h(mesh):
    min_h = float('inf')
    if mesh.dim == 2:
        # Loop over all 2D elements
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            
            circumradius = circumradius_2D(a, b, c)
            min_h = min(min_h, circumradius)

    elif mesh.dim == 3:
        # Loop over all 3D elements
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            try:
                circumradius = circumradius_3D(vertices)
                min_h = min(min_h, circumradius)
            except ValueError:
                continue

    else:
        raise ValueError("Unsupported mesh dimension")

    return min_h


# Example usage with a 2D mesh (unit square)
maxh = 1
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
# Example usage with a 3D mesh (unit cube)
geo = CSGeometry()
box = OrthoBrick(Pnt(0, 0, 0), Pnt(1, 1, 1))
geo.Add(box)
mesh2 = Mesh(geo.GenerateMesh(maxh=maxh))

# Calculate characteristic length for 2D and 3D
h_2d = calc_h(mesh)
h_3d = calc_h(mesh2)

print(f"Characteristic length h in 2D: {h_2d}")
print(f"Characteristic length h in 3D: {h_3d}")

# # Test with two tetrahedra from unit cube
# vertices_tetra_1 = [(0,0,0), (1,0,0), (0,1,0), (0,0,1)]
# vertices_tetra_2 = [(1,0,0), (0,1,0), (0,0,1), (0,1,1)]

# print("Circumradius for Tetrahedron 1:", circumradius_3D(vertices_tetra_1))
# print("Circumradius for Tetrahedron 2:", circumradius_3D(vertices_tetra_2))

Characteristic length h in 2D: 0.7071067811865478
Characteristic length h in 3D: 0.7071067811865475
